# Bay Area Public Salaries

### Introduction
The motivation of the notebook is to compare graduate student assistant salaries across various counties and compare them to the state's salaries. For the purpose of this study, only data from public goverment records were used which can be found on the web under many names e.g. "Civic Service Pay Scale", "Job Salary Schedule", "Pay Schedule", etc.

**Note**: These salaries are *not* to be confused with "New Graduate Entry Level" salaries, rather these salaries are "Graduate Student Internship" levels.  
<br />

<center><img src="https://upload.wikimedia.org/wikipedia/commons/d/d8/California_Bay_Area_county_map_%28zoom%26color%29.svg" alt="bay area counties" width="250" height="250"></center>


**Hypothesis**: Counties of interest near Silicon Valley are at or exceed state salaries.
* San Francisco, San Mateo, Santa Clara, Alameda
* Santa Cruz which is south of San Mateo (grayed out on the map) was added for interest

### Import Libraries

In [113]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from IPython.display import display  # for displaying pandas dataframe

### Read in Data
**Alameda County**  
This data requires a good chunk of preprocessing before it can be used. The original file was a PDF that was converted to a "csv" via [online tool](https://www.zamzar.com/). There are some issues with the original data and the conversion that need to be addressed:
* <u>Original PDF Format</u>
  * The tabular data was split every on every page and follows a pattern.
    * 8 rows of irrelevant data, 28-29 rows of relevant job data, and repeat ...
    * Best way to remove irrelevant data is to `pd.dropna()` on 0th column, then keep rows that have `str.len <= 5` in 0th column.
  * Since the data was too long to fit onto a single row the data for a sinle job overflows to next row.
    * Ex: Row 7 and 8 contain job info for ACCESS Program Clinical Mgr.
    * Row 7 has two columns worth of data "JobCode" and "JobDescription".
    * Row 8 has nineteen columns worth of data including pay steps, min and max monthly salary, etc.
    * *Hint: After removing the irrelevant data, separate the dataframe into even and odd rows, then concat the rows.* 
* <u>CSV Conversion</u>
  * Last 3 columns got messed up in the conversion.
  * Column 17 marks if the job is "FLSA": X = yes, N = no. Notice how the jobs with "N" got mixed up with Column 16 "AnnualMax" salary e.g. row 13 with "72,306.00 N".
    * *FLSA stands for Fair Labor Standards Act, which is a federal law that sets minimum wage.*
  * Column 18 marks the standard hours for the job e.g. 80, 75, etc.
    * If the job is not FLSA then the standard hours ended up in column 17 instead of 18.


In [114]:
alameda_raw = pd.read_csv('data/Alameda_County_Pay_Schedule_2024-11-15.csv', header=None)
display(alameda_raw.head(14))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PeopleSoft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Report ID:,HXHRI003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JOBCODE SALARY REPORT,NaN,NaN,NaN,NaN,NaN,Page No.,1,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Run Date 11/16/2024,NaN,NaN,NaN
3,As Of Date: 11/15/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Run Time 00:29:05,NaN,NaN,NaN
4,Sorted By:,Job Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Approx,Approx,Approx,NaN,NaN,NaN
6,Jobcode/ Mgmt,NaN,Effective,Union Job,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comp,Monthly,Monthly,Annual,Std,NaN
7,Job Grp,Class,Date,Code,Family,Grd,Step 01,Step 02,Step 03,Step 04,Step 05,Step 06,Step 07,Freq,Min,Max,Max FLSA,Hrs,NaN
8,6517,ACCESS Program Clinical Mgr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,21,SM,05/12/2024,U15,120,R02,4930.4,NaN,NaN,NaN,6052,NaN,NaN,B,10682.53,13112.67,157352,X,80.0


**Extract Relevant Rows**

In [115]:
alameda = alameda_raw.dropna(subset=[0])                  # remove rows that have NaN in 0th column
alameda = alameda[alameda[0].astype(str).str.len() <= 5]  # keep rows that have str.len <= 5 in 0th column

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # df display options w/o truncation
#     display(alameda)

alameda.reset_index(drop=True, inplace=True)
display(alameda.head(8))
print(len(alameda))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,6517,ACCESS Program Clinical Mgr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21,SM,05/12/2024,U15,120,R02,4930.4,NaN,NaN,NaN,6052,NaN,NaN,B,10682.53,13112.67,157352,X,80.0
2,5142,ALL IN Physician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29,SM,12/25/2022,R45,485,T64,10184.8,NaN,NaN,NaN,12374.4,NaN,NaN,B,22067.07,26811.2,321734.4,X,80.0
4,1281,Absentee Voting Technician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,62,NM,07/07/2024,10,556,C66,2331.75,2439,2538,2661.75,2781,NaN,NaN,B,5052.13,6025.5,"72,306.00 N",75,NaN
6,0205N,Account Clerk Assist SAN TAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,64,NM,07/07/2024,39,NaN,O84,13.76,NaN,NaN,NaN,18.92,NaN,NaN,H,NaN,NaN,N,75,NaN


2982


**Fix Column 18 "Standard Hours"**

In [116]:
# Find all indices that have strings that are digits (number of hours) in column 17 and assign them to column 18
mask = alameda[17].astype(str).str.isdigit()  # convert col to type str since its a mixed type, otherwise isdigit() fails
temp = alameda[17][mask].rename(18)  # apply mask and rename column to match column for the update
display(temp)
alameda.update(temp)
display(alameda.head(8))

5       75
7       75
9       75
11      75
13      75
        ..
2973    75
2975    80
2977    75
2979    75
2981    75
Name: 18, Length: 1029, dtype: object

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,6517,ACCESS Program Clinical Mgr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21,SM,05/12/2024,U15,120,R02,4930.4,NaN,NaN,NaN,6052,NaN,NaN,B,10682.53,13112.67,157352,X,80.0
2,5142,ALL IN Physician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29,SM,12/25/2022,R45,485,T64,10184.8,NaN,NaN,NaN,12374.4,NaN,NaN,B,22067.07,26811.2,321734.4,X,80.0
4,1281,Absentee Voting Technician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,62,NM,07/07/2024,10,556,C66,2331.75,2439,2538,2661.75,2781,NaN,NaN,B,5052.13,6025.5,"72,306.00 N",75,75
6,0205N,Account Clerk Assist SAN TAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,64,NM,07/07/2024,39,NaN,O84,13.76,NaN,NaN,NaN,18.92,NaN,NaN,H,NaN,NaN,N,75,75


**Fix Column 16 "AnnualMax" Salary and Column 17 "FLSA"**  
Split Column 16 and put values into Column 16 and 17.

In [117]:
mask2 = alameda[16].astype(str).str.contains('\s')  # look for rows that have whitespace i.e. needs splitting
temp = alameda[16][mask2].str.split(' ', n=1, expand=True)  # two col df with new column names 0 and 1
temp.rename(columns={0:16, 1:17}, inplace=True)
display(temp)
alameda.update(temp)  # update alameda df with temp df where rows and cols match
display(alameda.head(8))

,16,17
5,"72,306.00",N
9,"65,656.50",N
13,"70,102.50",N
17,"106,496.00",N
19,"94,341.00",X
...,...,...
2973,"83,908.50",N
2975,"80,163.20",N
2977,"87,067.50",N
2979,"119,866.50",X


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,6517,ACCESS Program Clinical Mgr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21,SM,05/12/2024,U15,120,R02,4930.4,NaN,NaN,NaN,6052,NaN,NaN,B,10682.53,13112.67,157352,X,80.0
2,5142,ALL IN Physician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29,SM,12/25/2022,R45,485,T64,10184.8,NaN,NaN,NaN,12374.4,NaN,NaN,B,22067.07,26811.2,321734.4,X,80.0
4,1281,Absentee Voting Technician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,62,NM,07/07/2024,10,556,C66,2331.75,2439,2538,2661.75,2781,NaN,NaN,B,5052.13,6025.5,"72,306.00",N,75
6,0205N,Account Clerk Assist SAN TAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,64,NM,07/07/2024,39,NaN,O84,13.76,NaN,NaN,NaN,18.92,NaN,NaN,H,NaN,NaN,N,75,75


In [118]:
mask3 = alameda[16].isin(['N', 'X'])  # look for rows that contain N or X
temp = alameda[16][mask3].rename(17)
display(temp)
alameda.update(temp)  # update alameda df with temp df where rows and cols match
alameda[16][mask3] = np.nan  # replace rows with NaN
display(alameda.head(8))

7       N
11      N
15      N
39      N
41      N
       ..
2665    N
2843    N
2845    N
2853    N
2879    N
Name: 17, Length: 129, dtype: object

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,6517,ACCESS Program Clinical Mgr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21,SM,05/12/2024,U15,120,R02,4930.4,NaN,NaN,NaN,6052,NaN,NaN,B,10682.53,13112.67,157352,X,80.0
2,5142,ALL IN Physician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29,SM,12/25/2022,R45,485,T64,10184.8,NaN,NaN,NaN,12374.4,NaN,NaN,B,22067.07,26811.2,321734.4,X,80.0
4,1281,Absentee Voting Technician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,62,NM,07/07/2024,10,556,C66,2331.75,2439,2538,2661.75,2781,NaN,NaN,B,5052.13,6025.5,"72,306.00",N,75
6,0205N,Account Clerk Assist SAN TAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,64,NM,07/07/2024,39,NaN,O84,13.76,NaN,NaN,NaN,18.92,NaN,NaN,H,NaN,NaN,NaN,N,75


**Delete Irrelevant Rows before Concatenation**  
There are some jobs that overflow to 3 rows with the 3rd row being a note, see below row 708. Note these rows have _NaN_ in 0th column.

In [119]:
display(alameda.iloc[706:711, 0:7])

,0,1,2,3,4,5,6
706,5344,Communicable Disese Srv Prctnr,NaN,NaN,NaN,***,NaN
707,24,NM,07/07/2024,3,905,N14,4316.8
708,4915,"Communication Specialist, Z7",NaN,NaN,NaN,NaN,NaN
709,21,PA,06/23/2024,U61,NaN,P71,4611.2
710,1363,"Communications Manager,ACERA",NaN,NaN,NaN,NaN,NaN


In [120]:
alameda.dropna(subset=[0], inplace=True)  # remove rows that have NaN in 0th column
alameda.reset_index(drop=True, inplace=True)
display(alameda.iloc[706:711, 0:7])

,0,1,2,3,4,5,6
706,5344,Communicable Disese Srv Prctnr,NaN,NaN,NaN,***,NaN
707,24,NM,07/07/2024,3,905,N14,4316.8
708,4915,"Communication Specialist, Z7",NaN,NaN,NaN,NaN,NaN
709,21,PA,06/23/2024,U61,NaN,P71,4611.2
710,1363,"Communications Manager,ACERA",NaN,NaN,NaN,NaN,NaN


**Split Even and Odd Rows**

In [121]:
# Split into even and odd DataFrames
df_even = alameda.iloc[::2, [0,1]]  # Select even rows and only columns [0,1]
df_odd = alameda.iloc[1::2]  # Select odd rows
# Rename
df_even.rename(columns={0:'Code', 1:'Title'}, inplace=True)
df_odd.columns = ['Group', 'Class', 'Date', 'Union', 'Family','Grade', 'Step01',
                  'Step02', 'Step03', 'Step04', 'Step05', 'Step6', 'Step07',
                  'CompFreq', 'MonthlyMin', 'MonthlyMax', 'AnnualMax', 'FLSA', 'Hrs']
# Reset Index
df_even.reset_index(drop=True, inplace=True)
df_odd.reset_index(drop=True, inplace=True)
display(df_even.tail())
display(df_odd.tail())

,Code,Title
1486,1467,Workforce Services Tech
1487,9410,Yardworker
1488,6511,Youth and Fam Srvs Prg Asst
1489,6498,Youth and Fam Srvs Therapist
1490,6499,Youth&Family Srv Lead Therap


,Group,Class,Date,Union,Family,Grade,Step01,Step02,Step03,Step04,Step05,Step6,Step07,CompFreq,MonthlyMin,MonthlyMax,AnnualMax,FLSA,Hrs
1486,34,NM,07/07/2024,6,688,R16,2700,2820,2944.5,3088.5,3227.25,NaN,NaN,B,5850,6992.38,"83,908.50",N,75
1487,85,NM,02/04/2024,2,796,M36,NaN,NaN,NaN,NaN,3083.2,NaN,NaN,B,6680.27,6680.27,"80,163.20",N,80
1488,54,NM,07/07/2024,8,828,T10,2814.75,2935.5,3071.25,3212.25,3348.75,NaN,NaN,B,6098.63,7255.63,"87,067.50",N,75
1489,24,NM,07/07/2024,5,900,O49,NaN,4000.5,4209.75,4408.5,4610.25,NaN,NaN,B,8667.75,9988.88,"119,866.50",X,75
1490,24,NM,07/07/2024,5,900,O50,NaN,4116.75,4332,4537.5,4745.25,NaN,NaN,B,8919.63,10281.38,"123,376.50",X,75


**Concatenate Dataframes**

In [122]:
assert len(df_even) == len(df_odd)
alameda = pd.concat([df_even, df_odd], axis=1)
with pd.option_context('display.max_columns', None):  # df display options w/o column truncation
    display(alameda)

,Code,Title,Group,Class,Date,Union,Family,Grade,Step01,Step02,Step03,Step04,Step05,Step6,Step07,CompFreq,MonthlyMin,MonthlyMax,AnnualMax,FLSA,Hrs
0,6517,ACCESS Program Clinical Mgr,21,SM,05/12/2024,U15,120,R02,4930.4,NaN,NaN,NaN,6052,NaN,NaN,B,10682.53,13112.67,157352,X,80.0
1,5142,ALL IN Physician,29,SM,12/25/2022,R45,485,T64,10184.8,NaN,NaN,NaN,12374.4,NaN,NaN,B,22067.07,26811.2,321734.4,X,80.0
2,1281,Absentee Voting Technician,62,NM,07/07/2024,10,556,C66,2331.75,2439,2538,2661.75,2781,NaN,NaN,B,5052.13,6025.5,"72,306.00",N,75
3,0205N,Account Clerk Assist SAN TAP,64,NM,07/07/2024,39,NaN,O84,13.76,NaN,NaN,NaN,18.92,NaN,NaN,H,NaN,NaN,NaN,N,75
4,1305,Account Clerk I,64,NM,07/07/2024,10,K545,C74,2130,2217.75,2314.5,2405.25,2525.25,NaN,NaN,B,4615,5471.38,"65,656.50",N,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,1467,Workforce Services Tech,34,NM,07/07/2024,6,688,R16,2700,2820,2944.5,3088.5,3227.25,NaN,NaN,B,5850,6992.38,"83,908.50",N,75
1487,9410,Yardworker,85,NM,02/04/2024,2,796,M36,NaN,NaN,NaN,NaN,3083.2,NaN,NaN,B,6680.27,6680.27,"80,163.20",N,80
1488,6511,Youth and Fam Srvs Prg Asst,54,NM,07/07/2024,8,828,T10,2814.75,2935.5,3071.25,3212.25,3348.75,NaN,NaN,B,6098.63,7255.63,"87,067.50",N,75
1489,6498,Youth and Fam Srvs Therapist,24,NM,07/07/2024,5,900,O49,NaN,4000.5,4209.75,4408.5,4610.25,NaN,NaN,B,8667.75,9988.88,"119,866.50",X,75


**Final Touches**  
CompFreq can be one of four values:
* B = Bi-weekly (x2 per month)
* H = Flat Rate per Hour (limited to max Hrs bi-weekly)
* S = Flat Rate per Session or Meeting
  * _Remove these rows from table since a true annual salary cannot be calculated from this._
* D = Flat Rate per Day

Change dtypes of columns that are dollar amounts or hours to `float`. Also compute monthly and annual salary using the min and max rates between Step01 to Step07.

In [133]:
# display(alameda.dtypes)
# print(alameda.CompFreq.unique())

# Drop Jobs with CompFreq = 'S'
alameda = alameda[alameda.CompFreq != 'S']

step_cols = list(alameda.filter(like='Step').columns)  # list of column names with substring 'Step'
for col in alameda.filter(like='Step').columns:
    alameda[col] = pd.to_numeric(alameda[col], errors='coerce')  # convert to float and ignore NaN errors
alameda['Hrs'] = pd.to_numeric(alameda['Hrs'], errors='coerce')  # convert last column 'Hrs'

mask = alameda.AnnualMax.isna()  # find rows where AnnualMax is not calculated

# Define custom function for pd.df.apply
# def monthly(x, f):  # monthly calc: x = dataframe, f = function to use e.g. np.nanmin or np.nanmax
#     if x['CompFreq'] == 'H':
#         return f(x[step_cols], axis=1) * x['Hrs'] * 2
#     elif x['CompFreq'] == 'D':
#         return f(x[step_cols], axis=1) * 5 * 4
#     else:  # bi-weekly
#         return f(x[step_cols], axis=1) * 2

def monthly(x, f):  # monthly calc: x = dataframe, f = function to use e.g. np.nanmin or np.nanmax
    return x.f()

# print(np.nanmin(alameda[step_cols], axis=1))
# print(monthly(alameda, np.nanmin))

alameda['MonthlyMin'] = alameda[step_cols].apply(lambda x: monthly(x, min))
# alameda['MonthlyMax'] = alameda[step_cols].apply(lambda x: monthly(x, f=np.nanmax()))

# one liner for above
# alameda[['MonthlyMin','MonthlyMax']] = alameda[step_cols].apply(lambda x: pd.Series([monthly(x, np.nanmin),
#                                                                                      monthly(x, np.nanmax)]))

# with pd.option_context('display.max_columns', None):  # df display options w/o column truncation
#     display(alameda)

TypeError: min expected 1 argument, got 0

### References
**Websites.** The following includes all websites for the various salary infomation.
* Alameda County https://salaryordinance.alamedacountyca.gov/article-1/
* San Francisco County https://www.sf.gov/resource/2023/classification-and-compensation-data
* San Mateo County https://www.smcgov.org/hr/job-classification-table
* Santa Cruz County https://www2.santacruzcountyca.gov/personnel/salsched/salsched.asp
* Santa Clara County https://esa.santaclaracounty.gov/outside-organizations/human-resources/master-salary-ordinance-executive-leadership-salary-ordinance
* California State https://eservices.calhr.ca.gov/EnterpriseHRPublic/payscales/payscalesearch